In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_rl
%set_random_seed 12

In [2]:
%presentation_style

In [3]:
%load_latex_macros


$\newcommand{\vect}[1]{{\mathbf{\boldsymbol{#1}} }}$
$\newcommand{\amax}{{\text{argmax}}}$
$\newcommand{\P}{{\mathbb{P}}}$
$\newcommand{\E}{{\mathbb{E}}}$
$\newcommand{\R}{{\mathbb{R}}}$
$\newcommand{\Z}{{\mathbb{Z}}}$
$\newcommand{\N}{{\mathbb{N}}}$
$\newcommand{\C}{{\mathbb{C}}}$
$\newcommand{\abs}[1]{{ \left| #1 \right| }}$
$\newcommand{\simpl}[1]{{\Delta^{#1} }}$


In [ ]:
%autoreload

from training_rl.offline_rl.load_env_variables import load_env_variables
load_env_variables()

import os
import warnings
import gymnasium as gym
import torch

from training_rl.offline_rl.behavior_policies.behavior_policy_registry import BehaviorPolicyType
from training_rl.offline_rl.custom_envs.custom_2d_grid_env.obstacles_2D_grid_register import ObstacleTypes
from training_rl.offline_rl.custom_envs.custom_envs_registration import CustomEnv, RenderMode, register_grid_envs
from training_rl.offline_rl.custom_envs.utils import Grid2DInitialConfig, InitialConfigCustom2DGridEnvWrapper
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_combined_minari_dataset
from training_rl.offline_rl.offline_policies.offpolicy_rendering import offpolicy_rendering
from training_rl.offline_rl.offline_policies.policy_registry import PolicyName
from training_rl.offline_rl.offline_trainings.offline_training import offline_training
from training_rl.offline_rl.offline_trainings.policy_config_data_class import (
    TrainedPolicyConfig, get_trained_policy_path)
from training_rl.offline_rl.offline_trainings.restore_policy_model import restore_trained_offline_policy
from training_rl.offline_rl.visualizations.utils import get_state_action_data_and_policy_grid_distributions, snapshot_env
from training_rl.offline_rl.utils import load_buffer_minari, widget_list

warnings.filterwarnings("ignore")
register_grid_envs()

render_mode = RenderMode.RGB_ARRAY_LIST

<img src="_static/images/aai-institute-cover.svg" alt="Snow" style="width:100%;">
<div class="md-slide title"> Offline RL distributional shift exercises </div>

# Off-Policy Distributional Shift - I

## Exercise I:

In this exercise we have two datasets that tries to bring the agent from (3,0) to (7,7) .

<img src="_static/images/nb_95_env_image_1.png" alt="grid environment configuration" style="height:400px;">

One policy is suboptimal and the other is optimal.

I  - **Suboptimal expert policy**:  collect ~ 8000 steps

II - **expert policy**: collect ~ 1000 steps

(This could be a realistic situation where you could have only a few human expert data available and the rest of the data is collected through a far from optimal policy.)

**We will use in this example as off-policy RL policy, the Deep Q-Network (DQN) algorithm,** that we introduced in the online RL section. As the DQN agent cannot interact with the environment we will feed the collected data through a ReplyBuffer, similarly as we did before in the imitation learning example.


Exercise: (Give a look to the imitation learning exercise)

1 - **We will use the 8x8 grid environment with a vertical obstacle : ObstacleTypes.dqn_obstacle_8x8**.

2 - **The policies: behavior_8x8_eps_greedy_4_0_to_7_7 and behavior_8x8_deterministic_4_0_to_7_7)**

3 - **For the dataset collection give a look to the solution**, specifically to the points:

- Configure the two datasets
- Create Minari combined datasets 

as we will fuse the two datasets coming from different policies into a single one (as you should do in a real problem)

4 - **Do the training using as policy BehaviorPolicyType.dqn to have a felling of how good (or how bad) is an offpolicy algorithm to deal with collected data.**

**SOLUTION:**

**Create the environment**

In [ ]:
ENV_NAME = CustomEnv.Grid_2D_8x8_discrete

# Grid configuration
OBSTACLE = ObstacleTypes.dqn_obstacle_8x8
INITIAL_STATE = (3, 0)
FINAL_STATE = (7, 7)

env_2D_grid_initial_config = Grid2DInitialConfig(
    obstacles=OBSTACLE,
    initial_state=INITIAL_STATE,
    target_state=FINAL_STATE,
)

env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=render_mode), env_config=env_2D_grid_initial_config)
snapshot_env(env)

**Configure and rendering policies**

In [ ]:
BEHAVIOR_POLICY_I = BehaviorPolicyType.behavior_8x8_eps_greedy_4_0_to_7_7
DATA_SET_IDENTIFIER_I = "_suboptimal"
NUM_STEPS_I = 8000

BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_deterministic_4_0_to_7_7
DATA_SET_IDENTIFIER_II = "_expert"
NUM_STEPS_II = 1000


In [ ]:
select_policy_to_render = widget_list([BEHAVIOR_POLICY_I, BEHAVIOR_POLICY_II])

In [ ]:
offpolicy_rendering(
    env_or_env_name=ENV_NAME,
    render_mode=render_mode,
    behavior_policy_name=select_policy_to_render.value,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=100,
)

**Create single Minari dataset**

( give a look to **create_combined_minari_dataset(...)** ) 

In [ ]:
config_combined_data = create_combined_minari_dataset(
        env_name=ENV_NAME,
        dataset_identifiers = (DATA_SET_IDENTIFIER_I, DATA_SET_IDENTIFIER_II),
        num_collected_points = (NUM_STEPS_I, NUM_STEPS_II),
        behavior_policy_names = (BEHAVIOR_POLICY_I, BEHAVIOR_POLICY_II),
        combined_dataset_identifier = "combined_data_sets_random_walk",
        env_2d_grid_initial_config = env_2D_grid_initial_config,
)

**check state-action distribution**

In [ ]:
#Create Buffers with minari datasets
name_combined_dataset = config_combined_data.data_set_name

buffer_data = load_buffer_minari(name_combined_dataset)
data_size = len(buffer_data)

# Compute state-action data distribution
state_action_count_data, _ = get_state_action_data_and_policy_grid_distributions(buffer_data, env)
snapshot_env(env)

**offline policy training**

In [ ]:
POLICY_NAME = PolicyName.dqn

NAME_EXPERT_DATA = name_combined_dataset
# TrainedPolicyConfig is a handy object that will help us to deal with the policy configuration data.
offline_policy_config = TrainedPolicyConfig(
    name_expert_data=NAME_EXPERT_DATA,
    policy_name=POLICY_NAME,
    render_mode=render_mode,
    device="cpu"
)


In [ ]:
# Run the training

NUM_EPOCHS = 4
BATCH_SIZE = 128
STEP_PER_EPOCH = 0.7*data_size

# After every epoch we will collect some test statistics from the policy from NUMBER_TEST_ENVS independent envs.
NUMBER_TEST_ENVS = 1


offline_training(
    offline_policy_config=offline_policy_config,
    num_epochs = NUM_EPOCHS,
    number_test_envs=NUMBER_TEST_ENVS,
    step_per_epoch=STEP_PER_EPOCH,
    restore_training=False,
)

### Results

**Restore policy**

In [ ]:
POLICY_FILE = "policy_best_reward.pth"

# restore a policy with the same configuration as the one we trained.
policy = restore_trained_offline_policy(offline_policy_config)
# load the weights
name_expert_data = offline_policy_config.name_expert_data
log_name = os.path.join(name_expert_data, POLICY_NAME)
log_path = get_trained_policy_path(log_name)
policy.load_state_dict(torch.load(os.path.join(log_path, POLICY_FILE), map_location="cpu"))


**Policy visualization**

In [ ]:
#env_2D_grid_initial_config.obstacles = OBSTACLE.obst_free_8x8
#env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=render_mode), env_config=env_2D_grid_initial_config)

offpolicy_rendering(
    env_or_env_name=env,
    render_mode=render_mode,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=100,
    imitation_policy_sampling=False
)

**State-action policy distribution**

In [ ]:
NUM_EPISODES = 10 # as more episodes the better

# compute statistics
state_action_count_data, state_action_count_policy = \
    get_state_action_data_and_policy_grid_distributions(
    buffer_data,
    env,
    policy,
    num_episodes=NUM_EPISODES,
    logits_sampling=True,
)

snapshot_env(env)

## Exercise II

**1 - Train again the DQN but now with random data (i.e. suboptimal random policy  and increase the expert data too,
expert data ~ 30000).**

Is the policy better? Why?

Remove the walls and see what happens, i.e:

In [ ]:
env_2D_grid_initial_config.obstacles = ObstacleTypes.obst_free_8x8
env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=render_mode), env_config=env_2D_grid_initial_config)

offpolicy_rendering(
    env_or_env_name=env,
    render_mode=render_mode,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=100,
    imitation_policy_sampling=False
)

**1 - Train again the DQN but now with random data (i.e. suboptimal random policy  and increase the expert data too,
expert data ~ 30000).**

Is the policy better? Why?

Remove the walls and see what happens, i.e:

In [ ]:
env_2D_grid_initial_config.obstacles = ObstacleTypes.obst_free_8x8
env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=render_mode), env_config=env_2D_grid_initial_config)

offpolicy_rendering(
    env_or_env_name=env,
    render_mode=render_mode,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=100,
    imitation_policy_sampling=False
)